In [1]:
!pip install selenium

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 9.4/9.4 MB 57.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 400.2/400.2 kB 39.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 58.3/58.3 kB 7.4 MB/s eta 0:00:00


In [4]:
!apt-get update

Get:1 http://security.ubuntu.com/ubuntu jammy-security InRelease [110 kB]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,626 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Hit:4 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:5 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [119 kB]
Get:6 http://security.ubuntu.com/ubuntu jammy-security/universe amd64 Packages [993 kB]
Hit:7 https://ppa.launchpadcontent.net/c2d4u.team/c2d4u4.0+/ubuntu jammy InRelease
Get:8 http://security.ubuntu.com/ubuntu jammy-security/main amd64 Packages [966 kB]
Get:9 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [498 kB]
Get:10 http://archive.ubuntu.com/ubuntu jammy-backports InRelease [109 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 Packages [1,235 kB]
Hit:13 https://ppa.

In [5]:
!apt-get install -y chromium-browser

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following additional packages will be installed:
  apparmor libfuse3-3 liblzo2-2 snapd squashfs-tools systemd-hwe-hwdb udev
Suggested packages:
  apparmor-profiles-extra apparmor-utils fuse3 zenity | kdialog
The following NEW packages will be installed:
  apparmor chromium-browser libfuse3-3 liblzo2-2 snapd squashfs-tools
  systemd-hwe-hwdb udev
0 upgraded, 8 newly installed, 0 to remove and 28 not upgraded.
Need to get 26.3 MB of archives.
After this operation, 116 MB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 apparmor amd64 3.0.4-2ubuntu2.2 [595 kB]
Get:2 http://archive.ubuntu.com/ubuntu jammy/main amd64 liblzo2-2 amd64 2.10-2build3 [53.7 kB]
Get:3 http://archive.ubuntu.com/ubuntu jammy/main amd64 squashfs-tools amd64 1:4.5-3build1 [159 kB]
Get:4 http://archive.ubuntu.com/ubuntu jammy-updates/main amd64 udev amd64 249.11-0ubu

In [6]:
!apt install chromium-chromedriver

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
The following NEW packages will be installed:
  chromium-chromedriver
0 upgraded, 1 newly installed, 0 to remove and 28 not upgraded.
Need to get 2,308 B of archives.
After this operation, 77.8 kB of additional disk space will be used.
Get:1 http://archive.ubuntu.com/ubuntu jammy-updates/universe amd64 chromium-chromedriver amd64 1:85.0.4183.83-0ubuntu2.22.04.1 [2,308 B]
Fetched 2,308 B in 0s (8,875 B/s)
Selecting previously unselected package chromium-chromedriver.
(Reading database ... 121361 files and directories currently installed.)
Preparing to unpack .../chromium-chromedriver_1%3a85.0.4183.83-0ubuntu2.22.04.1_amd64.deb ...
Unpacking chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...
Setting up chromium-chromedriver (1:85.0.4183.83-0ubuntu2.22.04.1) ...


In [8]:
!ls /usr/lib/chromium-browser

chromedriver


In [35]:
import requests
import time
import csv
import pandas as pd

from bs4 import BeautifulSoup
from datetime import datetime

from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.chrome.service import Service
from selenium.webdriver.common.keys import Keys

In [11]:
def driversetup():
    options = webdriver.ChromeOptions()
    #run Selenium in headless mode
    options.add_argument('--headless')
    options.add_argument('--no-sandbox')
    #overcome limited resource problems
    options.add_argument('--disable-dev-shm-usage')
    options.add_argument("lang=en")
    #open Browser in maximized mode
    options.add_argument("start-maximized")
    #disable infobars
    options.add_argument("disable-infobars")
    #disable extension
    options.add_argument("--disable-extensions")
    options.add_argument("--incognito")
    options.add_argument("--disable-blink-features=AutomationControlled")

    driver = webdriver.Chrome(options=options)

    driver.execute_script("Object.defineProperty(navigator, 'webdriver', {get: () => undefined});")

    return driver

In [15]:
def pagesource(url, driver):
    driver = driver
    driver.get(url)
    time.sleep(5)
    soup = BeautifulSoup(driver.page_source)
    driver.close()
    return soup

In [17]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
soup= pagesource (url, driver)
print(soup.prettify())

<html lang="es">
 <head data-app-path="/bme-exchange" data-hora-act="Thu, 14 Sep 2023 12:21:20 GMT" data-idioma="es">
  <meta charset="utf-8"/>
  <meta content="IE=edge" http-equiv="X-UA-Compatible"/>
  <meta content="width=device-width, initial-scale=1" name="viewport"/>
  <meta content="text/html; charset=utf-8" http-equiv="Content-Type"/>
  <meta content="Copyright © BME 2023" id="copyright" name="copyright"/>
  <meta content="BME Exchange" property="og:site_name"/>
  <meta content="es" property="og:locale"/>
  <meta content="website" property="og:type"/>
  <meta content="summary_large_image" property="twitter:card"/>
  <meta content="@BolsaBME" property="twitter:site"/>
  <meta content="Resumen de Índices | BME Exchange" property="og:title"/>
  <meta content="Resumen de Índices | BME Exchange" property="twitter:title"/>
  <meta content="https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen" property="og:url"/>
  <meta content="https://www.bolsasymercados.es/bme-exchange/es

In [37]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
driver.get(url)
time.sleep(5)
table= driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[1]")
#print(table)

for row in table:
  print(row.text)


In [39]:
tabla.head()

,Indice,Ultimo,dif,Maximo,Minimo,Fecha,Hora,Dif_año
0,IBEX 35®,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,IBEX 35® con Dividendos,NaN,NaN,NaN,NaN,NaN,NaN,NaN
2,IBEX MEDIUM CAP®,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,IBEX SMALL CAP®,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,IBEX 35® Bancos,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [52]:
driver = driversetup()
url = 'https://www.bolsasymercados.es/bme-exchange/es/Indices/Resumen'
driver.get(url)
time.sleep(5)

indices=[indice.text for indice in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[1]") ]
ultimos=[ultimo.text for ultimo in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[2]") ]
diferencias=[dif.text for dif in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[3]") ]
maximos=[maxim.text for maxim in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[4]") ]
minimos=[minim.text for minim in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[5]") ]
fechas=[fecha.text for fecha in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[6]") ]
horas=[hora.text for hora in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[7]") ]
diferencias_anyo=[dif_anyo.text for dif_anyo in driver.find_elements("xpath","//*[@id='root']/div[2]/table/tbody/tr/td[8]") ]

In [54]:
tabla=pd.DataFrame(columns=["Indice","Ultimo","dif","Maximo","Minimo","Fecha","Hora","Dif_año"])

tabla["Indice"]=indices
tabla["Ultimo"]=ultimos
tabla["dif"]=diferencias
tabla["Maximo"]=maximos
tabla["Minimo"]=minimos
tabla["Fecha"]=fechas
tabla["Hora"]=horas
tabla["Dif_año"]=diferencias_anyo

tabla.head()

,Indice,Ultimo,dif,Maximo,Minimo,Fecha,Hora,Dif_año
0,IBEX 35®,"9.451,00","0,29%","9.465,50","9.380,00",14/09/2023,13:18:53,"14,85%"
1,IBEX 35® con Dividendos,"31.225,80","0,29%","31.273,70","30.991,00",14/09/2023,13:18:53,"18,58%"
2,IBEX MEDIUM CAP®,"13.406,90","0,49%","13.407,70","13.344,60",14/09/2023,13:17:31,"4,75%"
3,IBEX SMALL CAP®,"7.862,00","-0,33%","7.905,20","7.833,40",14/09/2023,13:17:02,"9,42%"
4,IBEX 35® Bancos,"611,80","0,41%","613,00","604,20",14/09/2023,13:18:53,"16,87%"


In [58]:
tabla.to_csv("Ejemplo_scraping1.csv", encoding='utf-8')